# Benchmark Gehring & Homberger
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

While other notebooks such as [cvrptw_service_team_routing.ipynb](cvrptw_service_team_routing.ipynb) focus on the cuOpt API and high level problem modeling, here we focus on performance.

cuOpt offers a unique benefit over other solver_settings, specifically, time to solution.  In addition to achieving world class accuracy, cuOpt also produces these solutions in a time frame that allows for re-optimization in dynamic environments and rapid iteration over possible problem configurations.

Here we are demonstrating this performance on a large popular academic [dataset by Gehing & Homberger](https://www.sintef.no/projectweb/top/vrptw/homberger-benchmark/).  These problems are well studied and used as the basis for comparison for VRP research and product offerings. The particular instance we will test with is from the group of largest (1000 location) problems.  Each problem instance has an associated best known solution, the one we will measure against is shown below

**API Reference**: [cuOpt Server Documentation](https://docs.nvidia.com/cuopt/serv_api.html)

### Environment Setup
First, let's check if we have a GPU available in system. This is only applicable if you are running server through this notebook. Else comment the next cell.

In [ ]:
# Check for GPU
!nvidia-smi

#### Install dependencies


In [ ]:
# Install cuOpt

# Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

# !pip install --extra-index-url https://pypi.nvidia.com --user cuopt-server-cu12==25.5.* cuopt-sh-client==25.5.*

In [ ]:
#Install notebook dependencies
!pip install --user -q matplotlib scipy pandas numpy

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.spatial import distance
from cuopt_sh_client import CuOptServiceSelfHostClient

#### Run cuOpt server in background

Please disable the next cell in case you are trying to test against a server running on a different machine.

In [ ]:
import subprocess
import atexit
import signal
import os
import psutil
import requests
import time

host = "localhost"
port = "5000"

def is_cuopt_server_running():
    for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
        try:
            cmdline = proc.info['cmdline']
            if 'python3' in proc.info['name'] and cmdline:  # Check if cmdline exists and is not None
                if any('cuopt_server.cuopt_service' in cmd for cmd in cmdline):
                    return True
        except (psutil.NoSuchProcess, psutil.AccessDenied):
            pass
    return False

def wait_for_server_health(timeout=30):
    """Wait for server to be healthy, return True if healthy, False if timeout"""
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            response = requests.get(f'http://{host}:{port}/cuopt/health', timeout=5)
            if response.status_code == 200:
                print("Server is healthy!")
                return True
        except requests.exceptions.RequestException:
            pass
        
        print(f"Waiting for server to be ready... ({int(time.time() - start_time)}s elapsed)")
        time.sleep(1)
    
    print(f"Server health check timed out after {timeout} seconds")
    return False

# Only start server if it's not already running
if not is_cuopt_server_running():
    # Start cuOpt server in background
    server_process = subprocess.Popen(['python3', '-m', 'cuopt_server.cuopt_service', '-i', host, '-p', port],
                                    stdout=subprocess.PIPE,
                                    stderr=subprocess.PIPE)
    
    def cleanup_server():
        try:
            server_process.terminate()
            server_process.wait(timeout=5)  # Wait up to 5 seconds for graceful termination
            print("Terminated cuOpt server")
        except (subprocess.TimeoutExpired, ProcessLookupError):
            # If graceful termination fails, force kill
            try:
                server_process.kill()
                print("Killed cuOpt server")
            except ProcessLookupError:
                pass

    atexit.register(cleanup_server)
    print("Started cuOpt server")
else:
    print("cuOpt server is already running")

if not wait_for_server_health():
        print("Warning: Server may not be fully ready")


In [ ]:
#### Download the data
# Download benchmark data
# Create data directory if it doesn't exist
!rm -rf /tmp/data
!mkdir -p /tmp/data
!wget https://www.sintef.no/globalassets/project/top/vrptw/homberger/1000/homberger_1000_customer_instances.zip -O /tmp/data/homberger_data.zip

# Unzip the data
!unzip /tmp/data/homberger_data.zip -d /tmp/data/

# Get the file path
homberger_1000_file = '/tmp/data/C1_10_1.TXT'

# Check if the file exists
homberger_1000_file = '/tmp/data/C1_10_1.TXT'
if not os.path.exists(homberger_1000_file):
    raise FileNotFoundError(f"Could not find {homberger_1000_file}. Please check the path.")

best_known_solution = {
    "n_vehicles": 100,
    "cost": 42478.95
}

### Problem Data
The data for this problem instance are provided via text file.  cuOpt has a utility function available specifically for the Gehring & Homberger benchmark which converts the problem into the components required by cuOpt.

In [ ]:
def create_from_file(file_path, is_pdp=False):
    """
    Create a DataFrame from a problem file.
    
    Args:
        file_path: Path to the problem file
        is_pdp: Whether the file is a pickup and delivery problem
        
    Returns:
        Tuple of (df, vehicle_capacity, vehicle_num)
    """
    node_list = []
    with open(file_path, "rt") as f:
        count = 1
        for line in f:
            if is_pdp and count == 1:
                vehicle_num, vehicle_capacity, speed = line.split()
            elif not is_pdp and count == 5:
                vehicle_num, vehicle_capacity = line.split()
            elif is_pdp:
                node_list.append(line.split())
            elif count >= 10:
                node_list.append(line.split())
            count += 1

    vehicle_num = int(vehicle_num)
    vehicle_capacity = int(vehicle_capacity)
    
    columns = [
        "vertex",
        "xcord",
        "ycord",
        "demand",
        "earliest_time",
        "latest_time",
        "service_time",
        "pickup_index",
        "delivery_index",
    ]
    df = pd.DataFrame(columns=columns)

    for item in node_list:
        row = {
            "vertex": int(item[0]),
            "xcord": float(item[1]),
            "ycord": float(item[2]),
            "demand": int(item[3]),
            "earliest_time": int(item[4]),
            "latest_time": int(item[5]),
            "service_time": int(item[6]),
        }
        if is_pdp:
            row["pickup_index"] = int(item[7])
            row["delivery_index"] = int(item[8])
        df = pd.concat([df, pd.DataFrame(row, index=[0])], ignore_index=True)

    return df, vehicle_capacity, vehicle_num

In [ ]:
orders, vehicle_capacity, n_vehicles = create_from_file(homberger_1000_file)
n_locations = orders["demand"].shape[0]-1
print("Number of locations          : ", n_locations)
print("Number of vehicles available : ", n_vehicles)
print("Capacity of each vehicle     : ", vehicle_capacity)
print("\nInitial Orders information")
print(orders)

# Initialize cuOpt Service Client and cuOpt Problem Data

In [ ]:
client = CuOptServiceSelfHostClient(
    ip=host,
    port=port,
    polling_timeout=None
)

cuopt_problem_data = {}

### Cost Matrix

In [ ]:
coords = list(zip(orders['xcord'].to_list(),
                  orders['ycord'].to_list()))

cost_matrix = pd.DataFrame(distance.cdist(coords, coords, 'euclidean')).astype(np.float32).values.tolist()

### Set Cost Matrix

In [ ]:
cuopt_problem_data["cost_matrix_data"] = {
        "data": {
            "0": cost_matrix
        }
    }

### Set Fleet Data

In [ ]:
vehicle_locations = [[0, 0]] * n_vehicles

cuopt_problem_data["fleet_data"] = {
        "vehicle_locations": vehicle_locations,
        "capacities": [[vehicle_capacity] * n_vehicles]
}

### Set Task Data

In [ ]:
cuopt_problem_data["task_data"] = {
        "task_locations": orders['vertex'].values.tolist(),
        "demand": [orders['demand'].values.tolist()],
        "task_time_windows": [list(i) for i in list(zip(orders['earliest_time'].values.tolist(),
                                                        orders['latest_time'].values.tolist()))],
        "service_times": orders['service_time'].values.tolist()
}

### Helper functions to solve and process the output

In [ ]:
def solution_eval(vehicles, cost, best_known_solution):
    
    print(f"- cuOpt provides a solution using {vehicles} vehicles")
    print(f"- This represents {vehicles - best_known_solution['n_vehicles']} more than the best known solution")
    print(f"- Vehicle Percent Difference {(vehicles/best_known_solution['n_vehicles'] - 1)*100}% \n\n")
    print(f"- In addition cuOpt provides a solution cost of {cost}") 
    print(f"- Best known solution cost is {best_known_solution['cost']}")
    print(f"- Cost Percent Difference {(cost/best_known_solution['cost'] - 1)*100}%")

### Get Optimized Results

Update solver config and test different run-time 

**1 Minute Time Limit**

Note: due to the large amount of data network transfer time can exceed the requested solve time.


In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 60.0
    }

# Solve the problem
solver_response = client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

In [ ]:
# Evaluation:
solution_eval(solver_resp["num_vehicles"], solver_resp["solution_cost"], best_known_solution)

**2 Minute Time Limit**

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 120.0
    }

# Solve the problem
solver_response = client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

In [ ]:
# Evaluation:
solution_eval(solver_resp["num_vehicles"], solver_resp["solution_cost"], best_known_solution)

## License

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.